In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as ss
import matplotlib.pyplot as plt
from math import cos, sin, pi
from IPython.display import display, HTML, Javascript
from tune_sklearn import TuneGridSearchCV

%matplotlib inline
sns.set_style("ticks") 

In [2]:
df_protests = pd.read_csv('./saprotest.csv')
df_protests.head()

,ISSID,Province,Municipality_metro,Metro,Rural,TownCity_Name,Suburbareaplacename,Police_Station,Coordinates,Latitude,...,Cross_Street,Start_Date,End_Date,Ongoing,type,type_id,Violent_or_violent,violent,Rangecode,Reasonforprotest
0,105,9,Western Cape - City of Cape Town,1,0,Cape Town,Khayelitsha; Town Two phase four informal sett...,WC - Khayelitsha,"-34.034239,18.677953",-34.034238,...,Lansdowne Road,02/01/2013,02/12/2013,1,Service delivery,9,Violent,1,5,Protests during which people from other areas ...
1,4,1,Eastern Cape - Senqu,0,1,Sterkspruit,Sterkspruit,EC - Sterkspruit,"-30.524935,27.364588",-30.524934,...,Zastron road,01/01/2013,01/24/2013,1,Service delivery,9,Violent,1,1,The residents wanted Sterkspruit to be moved f...
2,67,6,KwaZulu Natal - eThekwini,1,0,Durban,"Dundee, Forbes & Manhattan Coal processing pla...",KZN - Dundee,"-28.170436,30.230345",-28.170435,...,Not applicable,01/17/2013,01/26/2013,1,Labour related,4,Violent,1,3,Decreased amount: the miners had initially bee...
3,730,3,Gauteng - City of Tshwane,1,0,Pretoria,Pretoria West - Lotus Gardens,GP - Pretoria West,"-25.751159,28.083004",-25.751159,...,Brizia Cres,01/03/2013,01/03/2013,0,Vigilantism,7,Violent,1,5,"Residents stoned the house, smashed windows an..."
4,62,3,Gauteng - City of Johannesburg,1,0,Johannesburg,"Johannesburg, Nelson Mandela bridge",GP - Johannesburg Central,"-26.128008,28.03176",-26.128008,...,Bompas,01/04/2013,01/04/2013,0,Crime related,1,Peaceful,0,3,The news of the death of South African mountai...


In [3]:
df_sentiment = df_protests[['Reasonforprotest','violent']]

In [4]:
from sklearn.model_selection import train_test_split,GridSearchCV, RandomizedSearchCV

X, X_Valid,Y,Y_Valid = train_test_split(df_sentiment['Reasonforprotest'].values, df_sentiment['violent'].values, test_size=0.25, random_state=None)

In [5]:
from text_processing import pre_process_corpus
train_reviews = [pre_process_corpus(text) for text in X]
val_reviews = [pre_process_corpus(text) for text in X_Valid]


In [6]:
#import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
tf.compat.v1.disable_resource_variables()
train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    {'sentence': np.array(train_reviews)}, Y, 
    batch_size=50, num_epochs=None, shuffle=True)
    
# Prediction on the whole training set.
predict_train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    {'sentence': np.array(train_reviews)}, Y, shuffle=False)
    
# Prediction on the whole validation set.
predict_val_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    {'sentence': np.array(val_reviews)}, Y_Valid, shuffle=False)
    


Instructions for updating:
non-resource variables are not supported in the long term



In [7]:
embedding_feature = hub.text_embedding_column(
    key='sentence', 
    module_spec="https://tfhub.dev/google/universal-sentence-encoder/2",
    trainable=False)

In [8]:
optimizer_adam = tf.compat.v1.train.AdamOptimizer(
    learning_rate=tf.compat.v1.train.exponential_decay(
        global_step=0,
        learning_rate=0.1,
        decay_steps=10000,
        decay_rate=0.96, staircase=True))

dnn = tf.estimator.DNNClassifier(
          hidden_units=[512, 128],
          feature_columns=[embedding_feature],
          n_classes=2,
          activation_fn=tf.nn.relu,
          dropout=0.1,
          optimizer=optimizer_adam)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/16/0h_tr4k15xvcvtfr6syc20w80000gn/T/tmptfbirv2k', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}

In [61]:
dnn = tf.estimator.DNNClassifier(
          hidden_units=[256, 128],
          feature_columns=[embedding_feature],
          n_classes=2,
          activation_fn=tf.nn.relu,
          dropout=0.3,
          optimizer=tf.train.AdamOptimizer(learning_rate=0.005))

In [62]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import time

TOTAL_STEPS = 500
STEP_SIZE = 100
for step in range(0, TOTAL_STEPS+1, STEP_SIZE):
    print()
    print('-'*100)
    print('Training for step =', step)
    start_time = time.time()
    dnn.train(input_fn=lambda: train_input_fn(), steps=STEP_SIZE)
    elapsed_time = time.time() - start_time
    print('Train Time (s):', elapsed_time)
    print('Eval Metrics (Train):', dnn.evaluate(input_fn=predict_train_input_fn))
    print('Eval Metrics (Validation):', dnn.evaluate(input_fn=predict_val_input_fn))


----------------------------------------------------------------------------------------------------
Training for step = 0
Train Time (s): 32.7277569770813
Eval Metrics (Train): {'accuracy': 0.847793, 'accuracy_baseline': 0.5540335, 'auc': 0.9453878, 'auc_precision_recall': 0.9342432, 'average_loss': 0.31393448, 'label/mean': 0.4459665, 'loss': 34.375824, 'precision': 0.8844622, 'prediction/mean': 0.39513904, 'recall': 0.75767916, 'global_step': 100}
Eval Metrics (Validation): {'accuracy': 0.7762557, 'accuracy_baseline': 0.53881276, 'auc': 0.8721682, 'auc_precision_recall': 0.84976554, 'average_loss': 0.49950156, 'label/mean': 0.4611872, 'loss': 54.69542, 'precision': 0.81707317, 'prediction/mean': 0.38345462, 'recall': 0.6633663, 'global_step': 100}

----------------------------------------------------------------------------------------------------
Training for step = 100
Train Time (s): 31.215128898620605
Eval Metrics (Train): {'accuracy': 0.9543379, 'accuracy_baseline': 0.5540335,

In [63]:
predict_example = tf.estimator.inputs.numpy_input_fn(
    {'sentence': np.array('Students protesting for higher education one citizen shot dead')}, 0, shuffle=False)

In [64]:
test = np.array(['Students peaceful protesting for better pay wage'])

In [65]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn(x={"sentence":test}, y=None, batch_size=1, num_epochs=None, shuffle=False)

In [66]:
pred = dnn.predict(predict_input_fn)

In [67]:
predictions_dict = next(pred)

In [68]:
predictions_dict

{'logits': array([-2.9135618], dtype=float32),
 'logistic': array([0.05148721], dtype=float32),
 'probabilities': array([0.9485128 , 0.05148721], dtype=float32),
 'class_ids': array([0]),
 'classes': array([b'0'], dtype=object),
 'all_class_ids': array([0, 1], dtype=int32),
 'all_classes': array([b'0', b'1'], dtype=object)}

In [31]:
print('kk')

kk


In [74]:
predict_test_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    {'sentence': np.array(['miners shot at Marikana,five dead','peacefull protest at the beach Cape Town','men protest for higher education, ten citizen killed by police','women die at protest against gender based violence get tear gassed by police several injured'])}, np.array([1,0,1,1]), shuffle=False)

In [75]:
print('Eval Metrics (Validation):', dnn.evaluate(input_fn=predict_test_input_fn))

Eval Metrics (Validation): {'accuracy': 0.5, 'accuracy_baseline': 0.75, 'auc': 0.8333326, 'auc_precision_recall': 0.958333, 'average_loss': 2.0154982, 'label/mean': 0.75, 'loss': 8.061993, 'precision': 1.0, 'prediction/mean': 0.2864325, 'recall': 0.33333334, 'global_step': 600}


In [76]:
pred = dnn.predict(predict_test_input_fn)

In [77]:
for p in pred:
    print('p:',p)

p: {'logits': array([13.555992], dtype=float32), 'logistic': array([0.99999875], dtype=float32), 'probabilities': array([1.296304e-06, 9.999987e-01], dtype=float32), 'class_ids': array([1]), 'classes': array([b'1'], dtype=object), 'all_class_ids': array([0, 1], dtype=int32), 'all_classes': array([b'0', b'1'], dtype=object)}
p: {'logits': array([-8.142369], dtype=float32), 'logistic': array([0.00029087], dtype=float32), 'probabilities': array([9.9970907e-01, 2.9086240e-04], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object), 'all_class_ids': array([0, 1], dtype=int32), 'all_classes': array([b'0', b'1'], dtype=object)}
p: {'logits': array([-1.7886477], dtype=float32), 'logistic': array([0.14323857], dtype=float32), 'probabilities': array([0.8567614, 0.1432386], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object), 'all_class_ids': array([0, 1], dtype=int32), 'all_classes': array([b'0', b'1'], dtype=object)}
p: {'logits': array([-6.1